In [1]:
%run helper_funcs.ipynb

In [2]:
import numpy as np
import cv2
import time 
import gluoncv as gcv
from keras.preprocessing.image import img_to_array

In [3]:
model = load_model('model1.h5')

In [ ]:
video = cv2.VideoCapture(0)
time.sleep(1)
while True:
    _,image = video.read()
    input_w, input_h = 416, 416
    shape_i = (input_w, input_h)
    image_w, image_h, layers = image.shape
    image_r = cv2.resize(image, shape_i)
    image_arr = img_to_array(image_r)
    image_arr = image_arr.astype('float32')
    image_arr /= 255.0
    image_arr = expand_dims(image_arr, 0)
    
    predicted_result = model.predict(image_arr)
    
    anchors = [[116,90, 156,198, 373,326], [30,61, 62,45, 59,119], [10,13, 16,30, 33,23]]

    threshold_val = 0.6
    boxes = list()
    for i in range(len(predicted_result)):
        # decode the output of the network
        boxes += get_network_output(predicted_result[i][0], anchors[i], threshold_val, input_h, input_w)
    
    new_w, new_h = input_w, input_h
    for i in range(len(boxes)):
        x_offset, x_scale = (input_w - new_w)/2./input_w, float(new_w)/input_w
        y_offset, y_scale = (input_h - new_h)/2./input_h, float(new_h)/input_h
        boxes[i].xmin = int((boxes[i].xmin - x_offset) / x_scale * image_w)
        boxes[i].xmax = int((boxes[i].xmax - x_offset) / x_scale * image_w)
        boxes[i].ymin = int((boxes[i].ymin - y_offset) / y_scale * image_h)
        boxes[i].ymax = int((boxes[i].ymax - y_offset) / y_scale * image_h)
        
    if len(boxes) > 0:
        nb_class = len(boxes[0].classes)
        for c in range(nb_class):
            sorted_data = np.argsort([-box.classes[c] for box in boxes])
            for i in range(len(sorted_data)):
                index_i = sorted_data[i]
                if boxes[index_i].classes[c] == 0: continue
                for j in range(i+1, len(sorted_data)):
                    index_j = sorted_data[j]
                    box1, box2 = boxes[index_i], boxes[index_j]
                    intersect_width = chk_for_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
                    intersect_height = chk_for_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])
                    intersect = intersect_width * intersect_height
                    w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
                    w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
                    union = w1*h1 + w2*h2 - intersect
                    if float(intersect) / union >= 0.5:
                        boxes[index_j].classes[c] = 0
    
    labels = ["person"]
    
    v_boxes, v_labels, v_scores = list(), list(), list()
    for box in boxes:
        for i in range(len(labels)):
            if box.classes[i] > threshold_val:
                v_boxes.append([box.ymin, box.xmin, box.ymax, box.xmax])
                v_labels.append(labels[i])
                v_scores.append(box.classes[i])
                
    num_people = len(v_boxes)
    if num_people == 1:
        text= '{} person detected.'.format(num_people)
    else:
        text = '{} people detected.'.format(num_people)
    print(text)
    color = (0,0,255)
    thickness=2
    fontScale=1
    org = (120,450)
    font = cv2.FONT_HERSHEY_SIMPLEX
    countme = cv2.putText(image, text, org, font, fontScale,color, thickness, cv2.LINE_AA, False) 
    
    class_ids,scores,bounding_boxes = [0]*num_people, v_scores, v_boxes
    print(scores)
    print(bounding_boxes)
    
    for i in range(len(bounding_boxes)):

        score = scores[i]
        if score > threshold_val:

            left = bounding_boxes[i][0]
            top = bounding_boxes[i][1]
            right = bounding_boxes[i][2]
            bottom = bounding_boxes[i][3]

            #draw a red rectangle around detected objects
            cv2.rectangle(image, (int(left), int(top)), (int(top), int(bottom)), (0, 0, 255), thickness=2)
    
#     preds = gcv.utils.viz.cv_plot_bbox(image,bounding_boxes,scores,class_ids,class_names = labels, scale=1)
    cv2.imshow('result',countme)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
video.release()
cv2.destroyAllWindows() 

0 people detected.
[]
[]
2 people detected.
[0.9982643, 0.98188883]
[[171, 152, 1335, 597], [300, -3, 1319, 195]]
2 people detected.
[0.9979621, 0.96954757]
[[162, 148, 1340, 602], [290, -3, 1320, 202]]
2 people detected.
[0.9984687, 0.9619786]
[[226, 193, 1306, 596], [289, -3, 1321, 207]]
2 people detected.
[0.99868315, 0.98072547]
[[184, 159, 1339, 595], [296, -3, 1313, 200]]
2 people detected.
[0.99786144, 0.9770967]
[[174, 154, 1333, 598], [304, -2, 1312, 201]]
2 people detected.
[0.99874735, 0.9869394]
[[187, 159, 1330, 592], [325, 1, 1301, 192]]
